# Control optimization in single qubit system (single parameter)

This is the source codes for the example discussed in Sec. IV A-D in Ref. [1].

The Hamiltonian of a controlled system can be written as
\begin{align}
H = H_0(\textbf{x})+\sum_{k=1}^K u_k(t) H_k,
\end{align}

where $H_0(\textbf{x})$ is the free evolution Hamiltonian with unknown parameters $\textbf{x}$ and $H_k$ 
represents the $k$th control Hamiltonian with $u_k$ the corresponding control coefficient.

In this example, the free evolution Hamiltonian of a single qubit system is $H_0 = \frac{1}{2}\omega \sigma_3$ with 
$\omega$ the frequency and $\sigma_3$ a Pauli matrix. The dynamics of the system is governed by
\begin{align}
\partial_t\rho=-i[H_0, \rho]+ \gamma_{+}\left(\sigma_{+}\rho\sigma_{-}-\frac{1}{2}\{\sigma_{-}\sigma_{+},\rho\}\right)+ \gamma_{-}\left(\sigma_{-}\rho\sigma_{+}-\frac{1}{2}\{\sigma_{+}\sigma_{-},\rho\}\right),
\end{align}

where $\gamma_{+}$, $\gamma_{-}$ are decay rates and $\sigma_{\pm}=(\sigma_1 \pm \sigma_2)/2$. The control Hamiltonian 
\begin{align}
H_\mathrm{c}=u_1(t)\sigma_1+u_2(t)\sigma_2+u_3(t)\sigma_3.
\end{align}

Here $\sigma_{1}$, $\sigma_{2}$ are also Pauli matrices. The probe state is taken as $|+\rangle$ and the measurement for CFI is $\{|+\rangle\langle+|, |-\rangle\langle-|\}$ with
$|\pm\rangle:=\frac{1}{\sqrt{2}}(|0\rangle\pm|1\rangle)$. Here $|0\rangle(|1\rangle)$ is the eigenstate of $\sigma_3$ with respect to the eigenvalue $1(-1)$.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2205.15588.

In [ ]:
from quanestimation import *
import numpy as np

# initial state
rho0 = 0.5*np.array([[1., 1.], [1., 1.]])
# free Hamiltonian
omega = 1.0
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
H0 = 0.5*omega*sz
# derivative of the free Hamiltonian on omega
dH = [0.5*sz]
# control Hamiltonians 
Hc = [sx, sy, sz]
# dissipation
sp = np.array([[0., 1.], [0., 0.]])  
sm = np.array([[0., 0.], [1., 0.]]) 
decay = [[sp, 0.], [sm, 0.1]]
# measurement
M1 = 0.5*np.array([[1., 1.], [1., 1.]])
M2 = 0.5*np.array([[1., -1.], [-1., 1.]])
M = [M1, M2]
# time length for the evolution
tspan = np.linspace(0., 10., 2500)
# guessed control coefficients
cnum = len(tspan)-1
ctrl0 = [np.array([np.zeros(cnum), np.zeros(cnum), np.zeros(cnum)])]

## Choose the control algorithm

In [ ]:
# control algorithm: auto-GRAPE
GRAPE_paras = {"Adam":True, "ctrl0":ctrl0, "max_episode":300, \
               "epsilon":0.01, "beta1":0.90, "beta2":0.99}
control = ControlOpt(savefile=False, method="auto-GRAPE", **GRAPE_paras)

# # control algorithm: GRAPE
# GRAPE_paras = {"Adam":True, "ctrl0":ctrl0, "max_episode":300, \
#               "epsilon":0.01, "beta1":0.90, "beta2":0.99}
# control = ControlOpt(savefile=False, method="GRAPE", **GRAPE_paras)

# # control algorithm: PSO
# PSO_paras = {"p_num":10, "ctrl0":ctrl0, "max_episode":[1000,100], \
#              "c0":1.0, "c1":2.0, "c2":2.0, "seed":1234}
# control = ControlOpt(savefile=False, method="PSO", **PSO_paras)

# # control algorithm: DE
# DE_paras = {"p_num":10, "ctrl0":ctrl0, "max_episode":1000, "c":1.0, \
#             "cr":0.5, "seed":1234}
# control = ControlOpt(savefile=False, method="DE", **DE_paras)


In [ ]:
# input the dynamics data
control.dynamics(tspan, rho0, H0, dH, Hc, decay=decay, ctrl_bound=[-2.0, 2.0], dyn_method="expm")

## Choose the objective function

In [ ]:
# objective function: QFI
control.QFIM()

# # objective function: CFI
# control.CFIM(M=M)